In [38]:
import sys

sys.path.append("../../src/stong_core_cython")

In [39]:
import numpy as np
import pandas as pd
import hartree_fock
import basis_data
import importlib
import concurrent.futures
from scipy.optimize import minimize

importlib.reload(hartree_fock)
importlib.reload(basis_data)

from atom import *
from hartree_fock import *
from basis_data import *

In [40]:
df = pd.read_csv("../make_base/sto_3g.csv")

In [41]:
def solve(zetas, atom_symbol):
    atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
    molecule = Molecule(atoms)
    basis_data = {atom_symbol: create_basis_data(df, zetas, 4, 1)}
    hartreefock = HartreeFock(molecule, basis_data, max_iter=10000, tol=1e-10)
    hartreefock.scf()
    hartreefock.calculate_total_energy()
    results = hartreefock.get_results()
    return results, hartreefock


def get_energy(zetas, atom_symbol):
    results, _ = solve(zetas, atom_symbol)
    return results["energy"]

In [49]:
elements = [
    "H",
    "He",
    "Li",
    "Be",
    "B",
    "C",
    "N",
    "O",
    "F",
    "Ne",
    "Na",
    "Mg",
    "Al",
    "Si",
    "P",
    "S",
    "Cl",
    "Ar",
    "K",
    "Ca",
]
# elements = ["H"]
zetas = {}
results = {}
for i, element in enumerate(elements):
    print(element)
    if i == 0:
        result = minimize(
            get_energy,
            [5, 3, 2, 1],
            # [2.69, 0.75, 0.3],
            args=(element),
        )
    else:
        result = minimize(
            get_energy,
            result.x,
            args=(element),
            bounds=[(0, None), (0, None), (0, None), (0, None)],
        )
    # result = minimize(get_energy, [2.69, 0.75, 0.3], args=(element))
    results[element] = result
    zetas[element] = result.x
    print(zetas[element])
    df_zetas = pd.DataFrame(zetas)
    df_zetas.to_csv("zetas.csv", index=False)

H
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, p

In [48]:
results

{'H':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -0.49970524980288045
         x: [ 2.768e+00  3.858e+00  2.441e+00  1.613e+00]
       nit: 37
       jac: [ 3.353e-07  4.359e-07 -1.278e-06 -1.643e-06]
  hess_inv: [[ 8.593e+02  8.767e+02  4.445e+02  1.612e+02]
             [ 8.767e+02  1.452e+03  6.667e+02  2.334e+02]
             [ 4.445e+02  6.667e+02  3.372e+02  1.264e+02]
             [ 1.612e+02  2.334e+02  1.264e+02  6.111e+01]]
      nfev: 225
      njev: 45}